In [1]:
import os, math, collections, itertools

from tqdm import tqdm

import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.measure
import numpy as np

from utils import save, unpickle, pixel_range, preprocess, batch_run, scale
from deepexplain.tensorflow import DeepExplain
from classifiers import CIFAR_CNN
from trainers import Trainer
from attacks import GM

os.environ["CUDA_VISIBLE_DEVICES"]="2"

datadir = './CIFAR10_data/'
batches = [datadir + batch for batch in os.listdir(datadir)]

for i in tqdm(range(5)):
    batch = unpickle(batches[i])

    if i == 0:
        data = batch[b'data'].astype(np.float32)
        cifar = np.transpose(np.reshape(data, [-1,3,32,32]), [0,2,3,1])
        labels = batch[b'labels']
    else:
        data = batch[b'data'].astype(np.float32)
        cifar = np.concatenate((cifar, np.transpose(np.reshape(data, [-1,3,32,32]), [0,2,3,1])), axis=0)
        labels = np.concatenate((labels, batch[b'labels']), axis=0)

scaled_cifar = cifar / 127.5 - 1.0

test_batch = unpickle(batches[5])
cifar_test = np.transpose(np.reshape(test_batch[b'data'], [-1,3,32,32]), [0,2,3,1])
scaled_cifar_test = cifar_test / 127.5 - 1.0
labels_test = np.array(test_batch[b'labels'])

data_train = (scaled_cifar, labels)
data_test = (scaled_cifar_test, labels_test)

cifar_mean = np.mean(cifar, axis=(0,1,2))

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
100%|██████████| 5/5 [00:02<00:00,  2.11it/s]


## Accuracy

In [ ]:
def eval_accuracy(logdir):
    
    tf.reset_default_graph()
    
    sess = tf.InteractiveSession()
    
    DNN = CIFAR_CNN(logdir, tf.nn.relu)
    DNN.load(sess)
    
    gm = GM(DNN, eps=0.4, step_size=2, n_steps=40, norm='2', loss_type='xent')
    
    train_acc = DNN.evaluate(sess, data_train)
    test_acc = DNN.evaluate(sess, data_test)
    adv_acc = DNN.evaluate(sess, (gm.attack(sess, data_test, batch_size=500), data_test[1]))
    
    sess.close()
    
    res = collections.OrderedDict([('Train', train_acc), ('Test', test_acc), ('Adv', adv_acc)])
    
    savefile = logdir.replace('tf_logs', 'results/accuracy')[:-1] + '.pickle'
    savedir = '/'.join(savefile.split('/')[:-1])
    
    if not os.path.exists(savedir):
        
        os.makedirs(savedir)
    
    save(res, savefile)

## ROAR KAR

In [3]:
def nonzero_min(x):
    
    return np.min(np.abs(x[x != 0]))

def Canny(images):
    
    res = []
    
    for img in images:
        
        img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.uint8)
        edges = cv2.Canny(img_bw, 100, 200).reshape(1, 32, 32) + np.random.normal(loc=0.0, scale=0.1, size=[1, 32, 32])
        res.append(edges)
    
    return np.concatenate(res, axis=0)


def random_remove(images, percentile, keep=False):
    
    images = np.copy(images)
    
    mask = np.random.binomial(1, (100 - percentile) / 100, size=images.shape[:-1])
    
    if keep:
        images[mask == 1] = cifar_mean
    else:
        images[mask == 0] = cifar_mean
    
    return images


def remove(images, attributions, percentile, keep=False, random=False):
    '''
    images       : tensor of shape [N,H,W,C]
    attributions : tensor of shape [N,H,W]
    percentile   : scalar between 0 and 100, inclusive
    keep         : if true keep q percent; otherwise remove q percent
    '''
    
    images = np.copy(images)
    
    thresholds = np.percentile(attributions, 100 - percentile, axis=(1,2), keepdims=True)
    
    if keep:
        images[attributions < thresholds] = cifar_mean
    else:
        images[attributions > thresholds] = cifar_mean
    
    return images


def occlude_dataset(DNN, de, attribution, loss, percentiles, test=False, keep=False, random=False, batch_size=1000, savedir='./'):
    
    if test:
        Xs = cifar_test
        ys = labels_test
    else:
        Xs = cifar
        ys = labels
    
    total_batch = math.ceil(len(Xs) / batch_size)
    
    if not random:
    
        hmaps = []

        for i in tqdm(range(total_batch)):

            batch_xs = Xs[i*batch_size:(i+1)*batch_size]
            batch_xs_scaled = scale(batch_xs)

            if 'edge' in attribution:
                attrs = Canny(batch_xs)
            else:
                attrs = preprocess(de.explain(attribution, loss, DNN.X, batch_xs_scaled), 0, 100, use_abs=True)
            
            # Add small random noise so np.percentile works correctly
            attrs += np.random.uniform(low=0.0, high=1e-8, size=attrs.shape)
            
            hmaps.append(attrs)
        
        hmaps = np.concatenate(hmaps, axis=0)
    
    for percentile in tqdm(percentiles):
        
        dataset = []
        
        for i in range(total_batch):
            
            batch_xs, batch_ys = Xs[i*batch_size:(i+1)*batch_size], ys[i*batch_size:(i+1)*batch_size]
            
            if random:
                occluded_images = random_remove(batch_xs, percentile, keep)
            else:
                batch_attrs = hmaps[i * batch_size:(i + 1) * batch_size]
                occluded_images = remove(batch_xs, batch_attrs, percentile, keep)
            
            dataset.append(scale(occluded_images))
        
        save(np.concatenate(dataset, axis=0), savedir + '{}_{}_{}.pickle'.format('test' if test else 'train', attribution, percentile))


def eval_roar_kar(logdir, keep, loss_type='xent', train_only=False):
    
    def get_savedir():
        
        savedir = logdir.replace('tf_logs', 'KAR_' + loss_type.upper() if keep else 'ROAR_' + loss_type.upper())
        
        if not os.path.exists(savedir):
            
            os.makedirs(savedir)
        
        return savedir
    
    percentiles = [10, 30, 50, 70, 90]
    
    heavy_methods = ['SmoothGrad', 'IntegGrad', 'DeepLIFT']

    attribution_methods = [
#                            ('Random'           , 'zero'), 
#                            ('Saliency'         , 'saliency'), 
#                            ('Guided BP'        , 'guidedbp'), 
#                            ('SmoothGrad'       , 'smoothgrad'), 
                           ('Grad * Input'     , 'grad*input'), 
#                            ('IntegGrad'        , 'intgrad'), 
#                            ('DeepLIFT'         , 'deeplift')
                          ]

    attribution_methods = collections.OrderedDict(attribution_methods)
    
    if not train_only:
    
        tf.reset_default_graph()

        sess = tf.InteractiveSession()

        with DeepExplain(session=sess, graph=sess.graph) as de:

            DNN = CIFAR_CNN(logdir, activation=tf.nn.relu)
            DNN.load(sess)
            
            if loss_type == 'xent':
                loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes)), logits=DNN.logits)
            elif loss_type == 'cw':
                label_mask = tf.stop_gradient(tf.one_hot(DNN.yi, depth=DNN.n_classes, on_value=True, off_value=False))
                c_logit = tf.boolean_mask(DNN.logits, label_mask)
                w_logit = tf.reduce_max(tf.reshape(tf.boolean_mask(DNN.logits, tf.logical_not(label_mask)), shape=[-1,9]), axis=1)
                loss = w_logit - c_logit
            else:
                raise Exception('Unknown loss type')

            for k, v in attribution_methods.items():

                if k in heavy_methods:
                    batch_size = 2500
                else:
                    batch_size = 2500
                
                occlude_dataset(DNN, de, v, loss, percentiles, False, keep, k == 'Random', batch_size, get_savedir())
                occlude_dataset(DNN, de, v, loss, percentiles, True,  keep, k == 'Random', batch_size, get_savedir())

        sess.close()
    
    ress = collections.OrderedDict([(k, []) for k in attribution_methods.keys()])
    
    for _ in range(3):
        
        for k, v in attribution_methods.items():
            
            res = []

            for p in percentiles:
                
                occdir = get_savedir() + '{}_{}_{}.pickle'.format('{}', v, p)
                data_train = (unpickle(occdir.format('train')), labels)
                data_test = (unpickle(occdir.format('test')), labels_test)
                
                tf.reset_default_graph()
                
                DNN = CIFAR_CNN('tf_logs/exp3/ROAR_Loss/', activation=tf.nn.relu)
                
                sess = tf.InteractiveSession()
                sess.run(tf.global_variables_initializer())
                
                trainer = Trainer(sess, DNN, data_train)
                trainer.train(20, p_epochs=30)
                
                acc = DNN.evaluate(sess, data_test)
                
                print('{}{} | Accuracy : {:.5f}'.format(k, p, acc))
                
                res.append(acc)
                
                sess.close()
                
            ress[k].append(res)
    
    res_mean = {k: np.mean(v, axis=0) for k, v in ress.items()}
    
    savefile = logdir.replace('tf_logs', 'results/{}'.format('kar_' + loss_type if keep else 'roar_' + loss_type))[:-1] + '.pickle'
    savedir = '/'.join(savefile.split('/')[:-1])
    
    print('Saving results at ' + savefile)
    
    if not os.path.exists(savedir):
        
        os.makedirs(savedir)
    
    if not os.path.isfile(savefile):
    
        save(res_mean, savefile)
    
    else:
        
        res_prev = unpickle(savefile)
        
        for k, v in res_mean.items():
        
            res_prev[k] = v
        
        save(res_prev, savefile)

## Train

In [4]:
def params_maker(param_names, param_values):
    
    product = itertools.product(*param_values)
    params = [list(zip(param_names, p)) for p in product]
    
    return params

def logdir_maker(attack_params):
    
    return 'tf_logs/exp3/adv/GM/{}/'.format('_'.join([str(v) for k, v in attack_params]))

def run(attack_params=None, train_first=True, run_sensitivity=True, run_roar_kar=True, loss_type='xent', keep=False, rk_train_only=False):
    
    if attack_params:
        logdir = logdir_maker(attack_params)
    else:
        logdir = 'tf_logs/exp3/standard/'
    
    print(logdir)
    
    if train_first:
    
        tf.reset_default_graph()

        DNN = CIFAR_CNN(logdir, activation=tf.nn.relu, attack=GM, attack_params=dict(attack_params))

        sess = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())

        trainer = Trainer(sess, DNN, data_train)
        trainer.train(n_epochs=20, p_epochs=5)
        
        sess.close()
        
        eval_accuracy(logdir)
    
    if run_roar_kar: eval_roar_kar(logdir, keep, loss_type, rk_train_only)

In [5]:
run(train_first=False, run_roar_kar=True, loss_type='xent', keep=False)
run(train_first=False, run_roar_kar=True, loss_type='xent', keep=True)

tf_logs/exp4/standard/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/standard/model


100%|██████████| 5/5 [01:03<00:00, 12.73s/it]


Grad * Input10 | Accuracy : 0.66500
Grad * Input30 | Accuracy : 0.59860
Grad * Input50 | Accuracy : 0.56720
Grad * Input70 | Accuracy : 0.52940
Grad * Input90 | Accuracy : 0.43270
Grad * Input10 | Accuracy : 0.64340
Grad * Input30 | Accuracy : 0.59950
Grad * Input50 | Accuracy : 0.56640
Grad * Input70 | Accuracy : 0.51390
Grad * Input90 | Accuracy : 0.42670
Grad * Input10 | Accuracy : 0.65900
Grad * Input30 | Accuracy : 0.59360
Grad * Input50 | Accuracy : 0.55960
Grad * Input70 | Accuracy : 0.51410
Grad * Input90 | Accuracy : 0.42940
Saving results at results/roar_xent/exp4/standard.pickle
tf_logs/exp4/standard/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/standard/model


100%|██████████| 5/5 [00:34<00:00,  6.82s/it]


Grad * Input10 | Accuracy : 0.46640
Grad * Input30 | Accuracy : 0.56230
Grad * Input50 | Accuracy : 0.61770
Grad * Input70 | Accuracy : 0.65830
Grad * Input90 | Accuracy : 0.70310
Grad * Input10 | Accuracy : 0.46860
Grad * Input30 | Accuracy : 0.56220
Grad * Input50 | Accuracy : 0.63180
Grad * Input70 | Accuracy : 0.65420
Grad * Input90 | Accuracy : 0.69890
Grad * Input10 | Accuracy : 0.47550
Grad * Input30 | Accuracy : 0.57770
Grad * Input50 | Accuracy : 0.60810
Grad * Input70 | Accuracy : 0.65790
Grad * Input90 | Accuracy : 0.69680
Saving results at results/kar_xent/exp4/standard.pickle


In [6]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.08 * i, 2) for i in range(1,21)], [2], [40], ['2'], ['xent']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=False)

tf_logs/exp4/adv/GM/0.08_2_40_2_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.08_2_40_2_xent/model


100%|██████████| 5/5 [00:28<00:00,  5.71s/it]


Grad * Input10 | Accuracy : 0.64940
Grad * Input30 | Accuracy : 0.59950
Grad * Input50 | Accuracy : 0.55780
Grad * Input70 | Accuracy : 0.52110
Grad * Input90 | Accuracy : 0.44060
Grad * Input10 | Accuracy : 0.65850
Grad * Input30 | Accuracy : 0.60660
Grad * Input50 | Accuracy : 0.56970
Grad * Input70 | Accuracy : 0.52350
Grad * Input90 | Accuracy : 0.43430
Grad * Input10 | Accuracy : 0.64840
Grad * Input30 | Accuracy : 0.60440
Grad * Input50 | Accuracy : 0.55700
Grad * Input70 | Accuracy : 0.51310
Grad * Input90 | Accuracy : 0.44030
Saving results at results/roar_xent/exp4/adv/GM/0.08_2_40_2_xent.pickle
tf_logs/exp4/adv/GM/0.16_2_40_2_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.16_2_40_2_xent/model


100%|██████████| 5/5 [00:19<00:00,  3.84s/it]


Grad * Input10 | Accuracy : 0.65360
Grad * Input30 | Accuracy : 0.58770
Grad * Input50 | Accuracy : 0.55370
Grad * Input70 | Accuracy : 0.50850
Grad * Input90 | Accuracy : 0.42250
Grad * Input10 | Accuracy : 0.65180
Grad * Input30 | Accuracy : 0.59590
Grad * Input50 | Accuracy : 0.54910
Grad * Input70 | Accuracy : 0.51210
Grad * Input90 | Accuracy : 0.42680
Grad * Input10 | Accuracy : 0.64970
Grad * Input30 | Accuracy : 0.59460
Grad * Input50 | Accuracy : 0.55120
Grad * Input70 | Accuracy : 0.50610
Grad * Input90 | Accuracy : 0.42570
Saving results at results/roar_xent/exp4/adv/GM/0.16_2_40_2_xent.pickle
tf_logs/exp4/adv/GM/0.24_2_40_2_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.24_2_40_2_xent/model


...

Grad * Input10 | Accuracy : 0.65170
Grad * Input30 | Accuracy : 0.56350
Grad * Input50 | Accuracy : 0.52690
Grad * Input70 | Accuracy : 0.47060
Grad * Input90 | Accuracy : 0.39110
Grad * Input10 | Accuracy : 0.64350
Grad * Input30 | Accuracy : 0.57270
Grad * Input50 | Accuracy : 0.50840
Grad * Input70 | Accuracy : 0.46690
Grad * Input90 | Accuracy : 0.39510
Grad * Input10 | Accuracy : 0.63860
Grad * Input30 | Accuracy : 0.56630
Grad * Input50 | Accuracy : 0.52310
Grad * Input70 | Accuracy : 0.45780
Grad * Input90 | Accuracy : 0.38800
Saving results at results/roar_xent/exp4/adv/GM/1.44_2_40_2_xent.pickle
tf_logs/exp4/adv/GM/1.52_2_40_2_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/1.52_2_40_2_xent/model


100%|██████████| 5/5 [00:49<00:00,  9.94s/it]


Grad * Input10 | Accuracy : 0.65120
Grad * Input30 | Accuracy : 0.56480
Grad * Input50 | Accuracy : 0.50670
Grad * Input70 | Accuracy : 0.45810
Grad * Input90 | Accuracy : 0.39720
Grad * Input10 | Accuracy : 0.63300
Grad * Input30 | Accuracy : 0.57000
Grad * Input50 | Accuracy : 0.51450
Grad * Input70 | Accuracy : 0.46240
Grad * Input90 | Accuracy : 0.39110
Grad * Input10 | Accuracy : 0.63840
Grad * Input30 | Accuracy : 0.55820
Grad * Input50 | Accuracy : 0.51480
Grad * Input70 | Accuracy : 0.47080
Grad * Input90 | Accuracy : 0.38960
Saving results at results/roar_xent/exp4/adv/GM/1.52_2_40_2_xent.pickle
tf_logs/exp4/adv/GM/1.6_2_40_2_xent/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/1.6_2_40_2_xent/model


100%|██████████| 5/5 [00:48<00:00,  9.72s/it]


Grad * Input10 | Accuracy : 0.63830
Grad * Input30 | Accuracy : 0.56720
Grad * Input50 | Accuracy : 0.50810
Grad * Input70 | Accuracy : 0.46130
Grad * Input90 | Accuracy : 0.39170
Grad * Input10 | Accuracy : 0.65190
Grad * Input30 | Accuracy : 0.56560
Grad * Input50 | Accuracy : 0.51270
Grad * Input70 | Accuracy : 0.46090
Grad * Input90 | Accuracy : 0.40090
Grad * Input10 | Accuracy : 0.65500
Grad * Input30 | Accuracy : 0.54650
Grad * Input50 | Accuracy : 0.52140
Grad * Input70 | Accuracy : 0.46540
Grad * Input90 | Accuracy : 0.39000
Saving results at results/roar_xent/exp4/adv/GM/1.6_2_40_2_xent.pickle


In [5]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.08 * i, 2) for i in range(6,21)], [2], [40], ['2'], ['cw']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=False)

tf_logs/exp4/adv/GM/0.48_2_40_2_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.48_2_40_2_cw/model


100%|██████████| 5/5 [00:27<00:00,  5.51s/it]


Grad * Input10 | Accuracy : 0.63720
Grad * Input30 | Accuracy : 0.57250
Grad * Input50 | Accuracy : 0.54010
Grad * Input70 | Accuracy : 0.48860
Grad * Input90 | Accuracy : 0.42030
Grad * Input10 | Accuracy : 0.64450
Grad * Input30 | Accuracy : 0.58500
Grad * Input50 | Accuracy : 0.54360
Grad * Input70 | Accuracy : 0.49440
Grad * Input90 | Accuracy : 0.41390
Grad * Input10 | Accuracy : 0.64020
Grad * Input30 | Accuracy : 0.59390
Grad * Input50 | Accuracy : 0.54580
Grad * Input70 | Accuracy : 0.48890
Grad * Input90 | Accuracy : 0.42660
Saving results at results/roar_xent/exp4/adv/GM/0.48_2_40_2_cw.pickle
tf_logs/exp4/adv/GM/0.56_2_40_2_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.56_2_40_2_cw/model


100%|██████████| 5/5 [00:27<00:00,  5.44s/it]


Grad * Input10 | Accuracy : 0.63910
Grad * Input30 | Accuracy : 0.58040
Grad * Input50 | Accuracy : 0.53860
Grad * Input70 | Accuracy : 0.49360
Grad * Input90 | Accuracy : 0.40250
Grad * Input10 | Accuracy : 0.64520
Grad * Input30 | Accuracy : 0.57800
Grad * Input50 | Accuracy : 0.53170
Grad * Input70 | Accuracy : 0.49790
Grad * Input90 | Accuracy : 0.39710
Grad * Input10 | Accuracy : 0.64030
Grad * Input30 | Accuracy : 0.57430
Grad * Input50 | Accuracy : 0.53610
Grad * Input70 | Accuracy : 0.48920
Grad * Input90 | Accuracy : 0.41110
Saving results at results/roar_xent/exp4/adv/GM/0.56_2_40_2_cw.pickle
tf_logs/exp4/adv/GM/0.64_2_40_2_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.64_2_40_2_cw/model


...

Grad * Input10 | Accuracy : 0.62790
Grad * Input30 | Accuracy : 0.55820
Grad * Input50 | Accuracy : 0.52090
Grad * Input70 | Accuracy : 0.46590
Grad * Input90 | Accuracy : 0.38780
Grad * Input10 | Accuracy : 0.64840
Grad * Input30 | Accuracy : 0.57010
Grad * Input50 | Accuracy : 0.51380
Grad * Input70 | Accuracy : 0.47230
Grad * Input90 | Accuracy : 0.38160
Grad * Input10 | Accuracy : 0.64360
Grad * Input30 | Accuracy : 0.55960
Grad * Input50 | Accuracy : 0.52750
Grad * Input70 | Accuracy : 0.47350
Grad * Input90 | Accuracy : 0.39030
Saving results at results/roar_xent/exp4/adv/GM/1.44_2_40_2_cw.pickle
tf_logs/exp4/adv/GM/1.52_2_40_2_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/1.52_2_40_2_cw/model


100%|██████████| 5/5 [00:29<00:00,  5.80s/it]


Grad * Input10 | Accuracy : 0.64960
Grad * Input30 | Accuracy : 0.56260
Grad * Input50 | Accuracy : 0.50130
Grad * Input70 | Accuracy : 0.48240
Grad * Input90 | Accuracy : 0.37780
Grad * Input10 | Accuracy : 0.64220
Grad * Input30 | Accuracy : 0.55920
Grad * Input50 | Accuracy : 0.51670
Grad * Input70 | Accuracy : 0.46450
Grad * Input90 | Accuracy : 0.40390
Grad * Input10 | Accuracy : 0.63660
Grad * Input30 | Accuracy : 0.56930
Grad * Input50 | Accuracy : 0.50160
Grad * Input70 | Accuracy : 0.46400
Grad * Input90 | Accuracy : 0.38210
Saving results at results/roar_xent/exp4/adv/GM/1.52_2_40_2_cw.pickle
tf_logs/exp4/adv/GM/1.6_2_40_2_cw/
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/1.6_2_40_2_cw/model


100%|██████████| 5/5 [00:33<00:00,  6.69s/it]


Grad * Input10 | Accuracy : 0.65590
Grad * Input30 | Accuracy : 0.57340
Grad * Input50 | Accuracy : 0.51670
Grad * Input70 | Accuracy : 0.48310
Grad * Input90 | Accuracy : 0.38740
Grad * Input10 | Accuracy : 0.65810
Grad * Input30 | Accuracy : 0.56710
Grad * Input50 | Accuracy : 0.52510
Grad * Input70 | Accuracy : 0.47120
Grad * Input90 | Accuracy : 0.39050
Grad * Input10 | Accuracy : 0.64820
Grad * Input30 | Accuracy : 0.56490
Grad * Input50 | Accuracy : 0.51940
Grad * Input70 | Accuracy : 0.46930
Grad * Input90 | Accuracy : 0.38420
Saving results at results/roar_xent/exp4/adv/GM/1.6_2_40_2_cw.pickle


In [ ]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.08 * i, 2) for i in range(1,21)], [2], [40], ['2'], ['xent']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=True)

In [ ]:
attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[round(0.08 * i, 2) for i in range(1,21)], [2], [40], ['2'], ['cw']])

for attack_params in attack_paramss:
    
    run(attack_params, train_first=False, run_sensitivity=False, run_roar_kar=True, loss_type='xent', keep=True)